In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda [-h] [-V] command ...
conda: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


ModuleNotFoundError: No module named 'geopy'

In [48]:
import requests
import lxml.html as lh
import pandas as pd

In [173]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [174]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [175]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [176]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [177]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [178]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [179]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [180]:
df.rename(columns = {'Borough\n':'Borough'}, inplace = True) 

In [181]:
df.rename(columns = {'Postal Code\n':'Postal Code'}, inplace = True) 
df.rename(columns = {'Neighborhood\n':'Neighborhood'}, inplace = True) 

In [182]:
df.columns

Index(['Postal Code', 'Borough', 'Neighborhood'], dtype='object')

In [183]:
df['Borough'] = df['Borough'].str[0:-1]

In [184]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned,\n
1,M2A\n,Not assigned,\n
2,M3A\n,North York,Parkwoods\n
3,M4A\n,North York,Victoria Village\n
4,M5A\n,Downtown Toronto,"Regent Park, Harbourfront\n"


In [185]:
df['Postal Code'] = df['Postal Code'].str[0:-1]
df['Neighborhood'] = df['Neighborhood'].str[0:-1]

In [186]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [187]:
df = df[df['Borough'] != 'Not assigned'] 

In [188]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [189]:
df['latitude'] = df.apply((lambda row: 0), axis = 1)

In [190]:
df['longitude'] = df.apply((lambda row: 0), axis = 1)

In [191]:
url = "http://cocl.us/Geospatial_data"
df1 = pd.read_csv(url)

In [192]:
df.columns

Index(['Postal Code', 'Borough', 'Neighborhood', 'latitude', 'longitude'], dtype='object')

In [204]:
for i in range(len(df)) : 
    for j in range(len(df1)) :  
        if(df.iloc[i,0]==df1.iloc[j,0]):
            df.iloc[i,3]=df1.iloc[j,1]
            df.iloc[i,4]=df1.iloc[j,2]

In [205]:
df.head()

,Postal Code,Borough,Neighborhood,latitude,longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
